In [1]:
import pandas as pd

In [2]:
import gensim

In [3]:
df_read = pd.read_excel('source\homestay master.xlsx', header=0)
df_read = df_read[~df_read["ลักษณะที่พัก"].isnull()]

In [4]:
fix_features = df_read[["รหัส",'อาหารเช้า', 'ทำอาหารเองได้ภายใน', 'ทำอาหารเองได้ภายนอก', 'กางเต๊นท์',
       'พืนที่สวน', 'สัตว์เลี้ยงในบริเวณ', 'แหล่งน้ำ', 'สระว่ายน้ำ',
       'ที่จอดรถ','WiFi ในห้อง']]
fix_features.set_index(["รหัส"],inplace=True)

In [5]:
fix_features.head(3)

,อาหารเช้า,ทำอาหารเองได้ภายใน,ทำอาหารเองได้ภายนอก,กางเต๊นท์,พืนที่สวน,สัตว์เลี้ยงในบริเวณ,แหล่งน้ำ,สระว่ายน้ำ,ที่จอดรถ,WiFi ในห้อง
รหัส,,,,,,,,,,
PKG0001,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
PKG0002,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
PKG0004,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [6]:
df_filter = df_read.copy()
df_filter.set_index(["รหัส"],inplace=True)
df_filter = df_filter.fillna('n/a')

In [7]:
df_filter['Platform ขาย'] = df_filter['Platform ขาย'].apply(lambda x: x.replace(' ','').split(","))
df_filter['Platform Review'] = df_filter['Platform Review'].apply(lambda x: x.replace(' ','').split(","))
df_filter['ลักษณะที่พัก'] = df_filter['ลักษณะที่พัก'].apply(lambda x: x.replace(' ','').split(","))
df_filter['สไตล์'] = df_filter['สไตล์'].apply(lambda x: x.replace(' ','').split(","))
df_filter['จุดขายพิเศษ'] = df_filter['จุดขายพิเศษ'].apply(lambda x: x.replace(' ','').split(","))
df_filter['อรรถประโยชน์'] = df_filter['อรรถประโยชน์'].apply(lambda x: x.replace(' ','').split(","))

In [8]:
df_filter.head(3)

,ชื่ออสังหาริมทรัพย์ตัวอย่าง,Lat-Lon,ตำบล,โซน,ประเภท,Rating (Google Map),จำนวนคน Rate (GM),Platform ขาย,Platform Review,ราคาเริ่มต้น/คืน,...,กางเต๊นท์,พืนที่สวน,สัตว์เลี้ยงในบริเวณ,แหล่งน้ำ,สระว่ายน้ำ,ที่จอดรถ,อรรถประโยชน์,WiFi ในห้อง,จุดขายพิเศษ,Unnamed: 28
รหัส,,,,,,,,,,,,,,,,,,,,,
PKG0001,Angel Valley,"14.469428283417516, 101.57710441380485",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,4.9,18.0,[Airbnb],[ใกล้ไกล],15000.0,...,0.0,1.0,1.0,1.0,0.0,1.0,"[ATV, จักรยาน, BBQ, Clubhouse, คาราโอเกะ, พักร...",1.0,[n/a],n/a
PKG0002,Antani Home Khaoyai,"14.47240737839758, 101.57533605260043",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,4.6,26.0,"[Agoda, Booking, Tripadviser, Facebook]","[ชิลไปไหน, Familypool]",2000.0,...,0.0,1.0,0.0,1.0,1.0,1.0,"[พูลวิลล่า, Clubhouse, คาราโอเกะ, BBQ, หมูกระทะ]",1.0,[n/a],n/a
PKG0004,Barn House Pong Ta Long,"14.471865480259176, 101.58365880642997",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,5.0,1.0,"[Agoda, Booking, Bedandbreakfast]",[n/a],10000.0,...,0.0,1.0,0.0,0.0,0.0,1.0,"[ห้ามจัดปาร์ตี้, BBQ, หมูกระทะ]",0.0,[n/a],n/a


In [9]:
def countElem(df_in, fieldName):
    dictionary = gensim.corpora.Dictionary(df_in[fieldName])
    df_in[fieldName] = df_in[fieldName].map(dictionary.doc2bow)
    df_in[fieldName] = df_in[fieldName].apply(lambda x:[(dictionary[id_], frequence) for id_, frequence in x])
    return df_in

In [10]:
df_filter = countElem(df_filter, 'Platform ขาย')
df_filter = countElem(df_filter, 'Platform Review')
df_filter = countElem(df_filter, 'ลักษณะที่พัก')
df_filter = countElem(df_filter, 'สไตล์')
df_filter = countElem(df_filter, 'จุดขายพิเศษ')
df_filter = countElem(df_filter, 'อรรถประโยชน์')

In [11]:
df_filter.head(3)

,ชื่ออสังหาริมทรัพย์ตัวอย่าง,Lat-Lon,ตำบล,โซน,ประเภท,Rating (Google Map),จำนวนคน Rate (GM),Platform ขาย,Platform Review,ราคาเริ่มต้น/คืน,...,กางเต๊นท์,พืนที่สวน,สัตว์เลี้ยงในบริเวณ,แหล่งน้ำ,สระว่ายน้ำ,ที่จอดรถ,อรรถประโยชน์,WiFi ในห้อง,จุดขายพิเศษ,Unnamed: 28
รหัส,,,,,,,,,,,,,,,,,,,,,
PKG0001,Angel Valley,"14.469428283417516, 101.57710441380485",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,4.9,18.0,"[(Airbnb, 1)]","[(ใกล้ไกล, 1)]",15000.0,...,0.0,1.0,1.0,1.0,0.0,1.0,"[(ATV, 1), (BBQ, 1), (Clubhouse, 1), (คาราโอเก...",1.0,"[(n/a, 1)]",n/a
PKG0002,Antani Home Khaoyai,"14.47240737839758, 101.57533605260043",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,4.6,26.0,"[(Agoda, 1), (Booking, 1), (Facebook, 1), (Tri...","[(Familypool, 1), (ชิลไปไหน, 1)]",2000.0,...,0.0,1.0,0.0,1.0,1.0,1.0,"[(BBQ, 1), (Clubhouse, 1), (คาราโอเกะ, 1), (พู...",1.0,"[(n/a, 1)]",n/a
PKG0004,Barn House Pong Ta Long,"14.471865480259176, 101.58365880642997",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,5.0,1.0,"[(Agoda, 1), (Booking, 1), (Bedandbreakfast, 1)]","[(n/a, 1)]",10000.0,...,0.0,1.0,0.0,0.0,0.0,1.0,"[(BBQ, 1), (หมูกระทะ, 1), (ห้ามจัดปาร์ตี้, 1)]",0.0,"[(n/a, 1)]",n/a


In [12]:
def unpivotCount(df_in, fieldName):
    df_out = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_in[fieldName]], \
          axis=1, sort=False).fillna(0).T.set_index(df_in.index)
    #df_out.reset_index(inplace=True)
    return df_out

Generate ข้อมูลตาม Domain ที่สนใจ เตรียมเอาไปทำ Conceptual Map แยก

In [13]:
df_platform = unpivotCount(df_filter, 'Platform ขาย')
df_platform_review = unpivotCount(df_filter, 'Platform Review')
df_acc = unpivotCount(df_filter, 'ลักษณะที่พัก')
df_style = unpivotCount(df_filter, 'สไตล์')
df_highlight = unpivotCount(df_filter, 'จุดขายพิเศษ')
df_util = unpivotCount(df_filter, 'อรรถประโยชน์')

รวมข้อมูล กรณีทำ MDS รวบทุก Domain

In [14]:
df_all = pd.concat([df_platform, df_platform_review,df_acc,df_style,df_highlight,df_util,fix_features]). \
            groupby(['รหัส']).sum()

In [15]:
df_all = df_all.drop('n/a', axis=1)

In [16]:
df_all.T.head(3)

รหัส,PKG0001,PKG0002,PKG0004,PKG0006,PKG0007,PKG0008,PKG0009,PKG0010,PKG0011,PKG0012,...,PKG0197,PKG0200,PKG0202,PKG0203,PKG0204,PKG0205,PKG0207,PKG0209,PKG0300,PKG0550
Airbnb,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Agoda,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
Booking,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [17]:
# ใช้ PCA รวบองค์ประกอบเพื่อ EDA ต่อไป
df_all.T.to_excel('output\datarawmatrix.xlsx', index=True)
df_all.to_excel('output\datarawmatrix_ori.xlsx', index=True)

Mapping similarity

In [18]:
df_map = df_all.T

In [19]:
item_item_matrix = pd.DataFrame(index=df_map.columns,columns=df_map.columns).fillna(0)

In [20]:
def jaccard_index(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    a_in_b = slice_df[(slice_df.A>0)&(slice_df.B>0)]
    a_occur = slice_df[(slice_df.A>0)]
    b_occur = slice_df[(slice_df.B>0)]
    jacc_index = a_in_b.shape[0]/(a_occur.shape[0]+b_occur.shape[0]-a_in_b.shape[0])
    return jacc_index

In [21]:
# Co-occurrence Similarity
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        item_item_matrix.iloc[i,j] = jaccard_index(df_map,i,j)

In [22]:
item_item_matrix.head()

รหัส,PKG0001,PKG0002,PKG0004,PKG0006,PKG0007,PKG0008,PKG0009,PKG0010,PKG0011,PKG0012,...,PKG0197,PKG0200,PKG0202,PKG0203,PKG0204,PKG0205,PKG0207,PKG0209,PKG0300,PKG0550
รหัส,,,,,,,,,,,,,,,,,,,,,
PKG0001,1.000000,0.250000,0.208333,0.300000,0.259259,0.222222,0.181818,0.233333,0.225806,0.285714,...,0.222222,0.212121,0.130435,0.185185,0.100000,0.307692,0.160000,0.066667,0.296296,0.142857
PKG0002,0.250000,1.000000,0.290323,0.282051,0.363636,0.375000,0.351351,0.548387,0.484848,0.473684,...,0.294118,0.416667,0.156250,0.387097,0.257143,0.406250,0.290323,0.194444,0.314286,0.416667
PKG0004,0.208333,0.290323,1.000000,0.214286,0.318182,0.272727,0.172414,0.333333,0.320000,0.250000,...,0.120000,0.206897,0.050000,0.227273,0.166667,0.208333,0.200000,0.038462,0.200000,0.129032
PKG0006,0.300000,0.282051,0.214286,1.000000,0.444444,0.310345,0.375000,0.354839,0.303030,0.388889,...,0.266667,0.285714,0.192308,0.233333,0.151515,0.258065,0.214286,0.088235,0.379310,0.216216
PKG0007,0.259259,0.363636,0.318182,0.444444,1.000000,0.375000,0.300000,0.541667,0.461538,0.451613,...,0.269231,0.379310,0.181818,0.454545,0.178571,0.259259,0.318182,0.066667,0.346154,0.290323


In [23]:
item_item_matrix.to_excel('output\coocMatrix.xlsx', index=True)

Mapping similarity #2 (No transpose)

In [24]:
item_item_matrix = pd.DataFrame(index=df_all.columns,columns=df_all.columns).fillna(0)

In [25]:
# Co-occurrence Similarity
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        item_item_matrix.iloc[i,j] = jaccard_index(df_all,i,j)

In [26]:
item_item_matrix.head()

,Airbnb,Agoda,Booking,Facebook,Tripadviser,Bedandbreakfast,www,PriceLine,Instagram,Findhotel,...,สระว่ายน้ำปรับอุณหภูมิได้,อาหารเช้า,ทำอาหารเองได้ภายใน,ทำอาหารเองได้ภายนอก,กางเต๊นท์,พืนที่สวน,สัตว์เลี้ยงในบริเวณ,แหล่งน้ำ,ที่จอดรถ,WiFi ในห้อง
Airbnb,1.000000,0.039474,0.027778,0.067797,0.000000,0.000000,0.035088,0.000000,0.100000,0.000000,...,0.000000,0.046512,0.131579,0.094737,0.029851,0.076923,0.096774,0.085106,0.082645,0.086207
Agoda,0.039474,1.000000,0.333333,0.567797,0.101449,0.014493,0.372093,0.014493,0.065789,0.014493,...,0.014493,0.552083,0.146067,0.429825,0.280000,0.567797,0.120482,0.341463,0.557377,0.516393
Booking,0.027778,0.333333,1.000000,0.211864,0.172414,0.037037,0.225806,0.000000,0.083333,0.000000,...,0.037037,0.175824,0.111111,0.152381,0.075000,0.222222,0.085106,0.172414,0.223140,0.211864
Facebook,0.067797,0.567797,0.211864,1.000000,0.060345,0.000000,0.410256,0.008621,0.103448,0.008621,...,0.008621,0.647059,0.262712,0.750000,0.508621,0.901639,0.186441,0.330508,0.942623,0.917355
Tripadviser,0.000000,0.101449,0.172414,0.060345,1.000000,0.000000,0.056604,0.000000,0.000000,0.142857,...,0.000000,0.060976,0.081081,0.063158,0.064516,0.060345,0.033333,0.116279,0.057851,0.060345


In [27]:
# ใช้แสดง SNA
item_item_matrix.to_excel('output\coocMatrix1.xlsx', index=True)

Mapping dissimilarity

In [28]:
item_item_matrix_dissim = pd.DataFrame(index=df_map.columns,columns=df_map.columns).fillna(0)

In [29]:
# Co-occurrence dissimilarity (distance = 1-index) ใช้กับ MDS (อย่าใช้ similarity)
for i in range(0,len(item_item_matrix_dissim.columns)):
    for j in range(0,len(item_item_matrix_dissim.columns)):
        item_item_matrix_dissim.iloc[i,j] = 1-jaccard_index(df_map,i,j)

In [30]:
# ใช้แสดง SNA
item_item_matrix_dissim.to_excel('output\coocDissimMatrix.xlsx', index=True)

In [31]:
item_item_matrix_dissim.head()

รหัส,PKG0001,PKG0002,PKG0004,PKG0006,PKG0007,PKG0008,PKG0009,PKG0010,PKG0011,PKG0012,...,PKG0197,PKG0200,PKG0202,PKG0203,PKG0204,PKG0205,PKG0207,PKG0209,PKG0300,PKG0550
รหัส,,,,,,,,,,,,,,,,,,,,,
PKG0001,0.000000,0.750000,0.791667,0.700000,0.740741,0.777778,0.818182,0.766667,0.774194,0.714286,...,0.777778,0.787879,0.869565,0.814815,0.900000,0.692308,0.840000,0.933333,0.703704,0.857143
PKG0002,0.750000,0.000000,0.709677,0.717949,0.636364,0.625000,0.648649,0.451613,0.515152,0.526316,...,0.705882,0.583333,0.843750,0.612903,0.742857,0.593750,0.709677,0.805556,0.685714,0.583333
PKG0004,0.791667,0.709677,0.000000,0.785714,0.681818,0.727273,0.827586,0.666667,0.680000,0.750000,...,0.880000,0.793103,0.950000,0.772727,0.833333,0.791667,0.800000,0.961538,0.800000,0.870968
PKG0006,0.700000,0.717949,0.785714,0.000000,0.555556,0.689655,0.625000,0.645161,0.696970,0.611111,...,0.733333,0.714286,0.807692,0.766667,0.848485,0.741935,0.785714,0.911765,0.620690,0.783784
PKG0007,0.740741,0.636364,0.681818,0.555556,0.000000,0.625000,0.700000,0.458333,0.538462,0.548387,...,0.730769,0.620690,0.818182,0.545455,0.821429,0.740741,0.681818,0.933333,0.653846,0.709677
